In [1]:
# Import the necessary libraries
from bs4 import BeautifulSoup 
import requests
import numpy as np
import pandas as pd
import math

# THIS ASSIGNMENT IS FOR QUESTION 1

### STEP 1: Build the code to scrape the following Wikipedia page to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(URL)
html_soup=BeautifulSoup(response.text, 'html.parser')
response.status_code

200

In [3]:
postal_code = html_soup.find('table', class_ = "wikitable sortable")

In [4]:
len(postal_code)

2

### STEP 2: Manipulate the html parser and get all the necessary information in the format we need

In [5]:
Postcode= []
Borough= []
Neighbourhood = []

for tr in postal_code.find_all('tr'):

    for idx, td in enumerate(tr.find_all('td')):                
        #Postcode.append(td[0].text)
        if idx == 0:
            if td.text =="Not assigned":
                Postcode.append(np.NaN)
            else:
                Postcode.append(td.text)
            #print(td.text)
        elif idx== 1:
            if td.text =="Not assigned":
                Borough.append(np.NaN)
            else:
                Borough.append(td.text)
            #print(td.text)
        elif idx == 2:
            #if td.text.strip() =="Not assigned":
            #    Neighbourhood.append(np.NaN)
            #else:
            Neighbourhood.append(td.text.strip())
            #print(td.text)
        #print(idx)


### STEP 3: Ensure the final output from the web scraping is in pandas dataframe

In [6]:
post=pd.DataFrame({
    "Postcode" : Postcode,
    "Borough": Borough,
    "Neighbourhood": Neighbourhood
                  }) 
post.head(n=15)

,Borough,Neighbourhood,Postcode
0,NaN,Not assigned,M1A
1,NaN,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,NaN,Not assigned,M8A


In [266]:
len(post)

289

### STEP 4 - Ensure all the conditions are met before finalizing the output

In [9]:
# Only process the cells that have an assigned Borough. Ignore cells with a Borough that is Not assigned.
post = post.loc[post["Borough"].dropna(axis=0, how='any').index.values,]
len(post)

212

In [10]:
post.sort_values(by=['Postcode', 'Borough'],inplace=True)

In [11]:
post[post["Neighbourhood"] == "Not assigned"]

,Borough,Neighbourhood,Postcode
8,Queen's Park,Not assigned,M7A


In [12]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
post.loc[post["Neighbourhood"] == "Not assigned", ["Neighbourhood"]] = post[post["Neighbourhood"] == "Not assigned"]["Borough"]

In [13]:
post[post["Neighbourhood"] == "Not assigned"]

,Borough,Neighbourhood,Postcode


In [14]:
post[post["Postcode"] == "M7A" ] 

,Borough,Neighbourhood,Postcode
8,Queen's Park,Queen's Park,M7A


In [15]:
post.head()

,Borough,Neighbourhood,Postcode
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B
27,Scarborough,Highland Creek,M1C
28,Scarborough,Rouge Hill,M1C
29,Scarborough,Port Union,M1C


In [16]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

post = pd.DataFrame(post.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)), index=None)
post.reset_index(inplace = True)                    

In [17]:
post.head(n=10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [18]:
post.shape

(103, 3)

## THIS IS THE END OF ASSIGNMENT FOR QUESTION 1

## STARTING FOR ASSIGNMENT IN QUESTION 2

### STEP 1 - READ THE DATA FROM THE URL

In [20]:
geo_post =  pd.read_csv("http://cocl.us/Geospatial_data")
geo_post.head(n=10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### STEP 2 - MERGE THE DATA TO THE FINAL DATA CREATED IN QUESTION NO 1 TO APPEND LAT/LONG INFORMATION

In [21]:
post_geo = pd.merge(left = post, right = geo_post, how ="inner", left_on= "Postcode", right_on ="Postal Code")

In [22]:
post_geo.dtypes

Postcode          object
Borough           object
Neighbourhood     object
Postal Code       object
Latitude         float64
Longitude        float64
dtype: object

In [23]:
post_geo.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [24]:
# USe pop function to remove the deuplicate columns
post_geo.pop("Postal Code")
post_geo.head(n=10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [25]:
post_geo.shape

(103, 5)

## THIS IS THE END OF ASSIGNMENT FOR QUESTION 2